In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
face_1 = np.load('with_mask.npy')
face_2 = np.load('without_mask.npy')

In [3]:
face_1.shape

(400, 50, 50, 3)

In [4]:
face_1 = face_1.reshape((400,-1))
face_2 = face_2.reshape((400,-1))

In [5]:
face_1.shape

(400, 7500)

In [6]:
faceData = np.concatenate([face_1,face_2])

In [83]:
faceData.shape

(800, 7500)

In [7]:
names = {0 : "Mask", 1 : "No Mask"}

In [8]:
labels = np.zeros((faceData.shape[0],1))

In [9]:
labels[400:,:] = 1.0

In [10]:
# def distance(x1,x2):
#     return np.sqrt(sum(x2 - x1) ** 2)

In [11]:
# def knn(data,target,k=5):
#     n = data.shape[0]
#     d = []
#     for i in range(n):
#         d.append(distance(data[i],target))
#     d = np.asarray(d)
#     indexes = np.argsort(d)
#     sorted_labels = labels[indexes][:k]
#     count = np.unique(sorted_labels,return_counts=True)
#     max_index = np.argmax(count[1])
#     result = count[0][max_index]
#     return result

In [12]:
dataset = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_COMPLEX

In [13]:
# try:
#     cap = cv2.VideoCapture(0)
#     while True:
#         flag, frame = cap.read()
#         if flag:
#             faces = dataset.detectMultiScale(frame)
#             for x,y,w,h in faces:
#                 cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),4)
#                 face = frame[y:y+h,x:x+w,:]
#                 face = cv2.resize(face,(50,50))
#                 index = knn(faceData,face.flatten())
#                 name = names[int(index)]
#                 cv2.putText(frame,name,(x,y),font,1,(244,100,0),1)
#             cv2.imshow('result',frame)
#             if cv2.waitKey(10) == ord('q'):
#                 break
#         else:
#             print("Camera not working")
# except BaseException as ex:
#     print("Exception",ex)
# finally:
#     cap.release()
#     cv2.destroyAllWindows()

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
x_train, x_test, y_train, y_test = train_test_split(faceData, labels, test_size = 0.25)

In [17]:
svm = SVC()
svm.fit(x_train, y_train.flatten())

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
y_pred_svm = svm.predict(x_test)

In [19]:
accuracy_score(y_test, y_pred_svm)

1.0

In [20]:
from sklearn.decomposition import PCA

In [21]:
comp = PCA(n_components=3)

In [22]:
# comp.fit(x_train)

In [23]:
x_train = comp.fit_transform(x_train)

In [24]:
svm = SVC()
svm.fit(x_train, y_train.flatten())

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [25]:
x_test = comp.fit_transform(x_test)

In [26]:
y_pred = svm.predict(x_test)

In [27]:
accuracy_score(y_test, y_pred)

1.0

In [28]:
try:
    cap = cv2.VideoCapture(0)
    while True:
        flag, frame = cap.read()
        if flag:
            faces = dataset.detectMultiScale(frame)
            for x,y,w,h in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),4)
                face = frame[y:y+h,x:x+w,:]
                face = cv2.resize(face,(50,50))
                face = face.reshape(1,-1)
                face = comp.transform(face)
                index = svm.predict(face)[0]
                name = names[int(index)]
                cv2.putText(frame,name,(x,y),font,1,(244,100,0),1)
            cv2.imshow('result',frame)
            if cv2.waitKey(10) == ord('q'):
                break
        else:
            print("Camera not working")
except BaseException as ex:
    print("Exception",ex)
finally:
    cap.release()
    cv2.destroyAllWindows()